# It is compulsory to run it with python  3.11

To make sure tensorflow is correct in Environment change it to python 3.11

In [1]:
# This should be the first block to be executed!
# Install flask to use it later

!pip install flask==2.3.3


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
from json import loads, dumps

### 2. Initialize an app/service using Flask
- We use Python Flask to create an app
- We add a base endpoint for the service by mapping the path "/"
- The base URL corresponds to the incoming connection of this notebook (see *7. Start Your App/Service below*)

In [5]:
from flask import Flask, jsonify, request

app = Flask(__name__)

@app.route("/")
def ehr_app():
    return "This is an Electronic Health Record service."



In [6]:
# Creating POST endpoint to add new patient
# The method expects a JSON object as input
# sample JSON: {"patient_id":1,"patient_fname":"eve","doctor":"Man Steiner","bp_val":95,"chol_val":100}


@app.route('/ArtidisMeasurement/<int:specimen_ID>', methods=['POST'])
def addArtidisDiagnosis(specimen_ID):
    df_artidis = pd.read_csv("artidis_result.csv")
    artidis_record = df_artidis.loc[df_artidis["Specimen_ID"] == specimen_ID]
    df_specimen = pd.read_csv("specimen_database3.csv")


    specimen_row = df_specimen[df_specimen['Specimen_ID'] == specimen_ID]
    specimen_row.loc[specimen_row['Specimen_ID'] == specimen_ID, 'ArtidisMeasurement'] = artidis_record["Diagnosis"].values[0]

    df_specimen.update(specimen_row)

    df_specimen.to_csv('specimen_database3.csv', index=False)

    
    specimen_record = df_specimen.loc[df_specimen["Specimen_ID"] == specimen_ID]
    result = specimen_record.to_json(orient="records")
    parsed= loads(result)

    obj = {
        "specimen": parsed
    }
    return obj, 201


In [7]:
#testing Add measurement

def addArtidisDiagnosis1(specimen_ID):
    
    df_artidis = pd.read_csv("artidis_result.csv")
    artidis_record = df_artidis.loc[df_artidis["Specimen_ID"] == specimen_ID]

    print(artidis_record["Diagnosis"].values[0])
    diagnosis01=artidis_record["Diagnosis"].values[0]
    print(diagnosis01)
    df_specimen = pd.read_csv("specimen_database3.csv")


    specimen_row = df_specimen[df_specimen['Specimen_ID'] == specimen_ID]
    specimen_row.loc[specimen_row['Specimen_ID'] == specimen_ID, 'ArtidisMeasurement'] = artidis_record["Diagnosis"].values[0]
    print(specimen_row)
    df_specimen.update(specimen_row)

    df_specimen.to_csv('specimen_database3.csv', index=False)

    
    specimen_record = df_specimen.loc[df_specimen["Specimen_ID"] == specimen_ID]
    result = specimen_record.to_json(orient="records")
    parsed= loads(result)

    obj = {
        "specimen": parsed
    }
    return obj, 201
#specimen_ID=10    
#addArtidisDiagnosis1(specimen_ID)

In [8]:

@app.route('/addspecimen', methods=['POST'])
def addSpecimen():
    if request.is_json:
        patient_rec = request.get_json()
        df_patient = pd.read_csv("specimen_database3.csv")

        df_patient_new = df_patient.append(patient_rec, ignore_index=True)
        df_patient_new.to_csv('specimen_database3.csv', index=False)

        obj = {
            "specimen": patient_rec
        }
        return obj, 201
    return {"error": "Request must be JSON"}, 415

In [9]:
@app.route('/addeligibility2', methods=['POST'])
def addEligibility2():
    if request.is_json:
        specimen_rec = request.get_json()
        df_specimen = pd.read_csv("specimen_database3.csv")
        specimen_ID = specimen_rec['Specimen_ID']
        eligibility_2 = specimen_rec['Eligibility_2']
        specimen_row = df_specimen[df_specimen['Specimen_ID'] == specimen_ID]
        specimen_row.loc[specimen_row['Specimen_ID'] == specimen_ID, 'Eligibility_2'] = eligibility_2
        df_specimen.update(specimen_row)

        df_specimen.to_csv('specimen_database3.csv', index=False)

        obj = {
            "specimen": specimen_rec
        }
        return obj, 201
    return {"error": "Request must be JSON"}, 415

In [10]:
def addEligibility2_2(eligibility_2):


    df_specimen = pd.read_csv("specimen_database3 (4).csv")
    
    specimen_ID=1
    specimen_row = df_specimen[df_specimen['Specimen_ID'] == specimen_ID]
    specimen_row.loc[specimen_row['Specimen_ID'] == specimen_ID, 'Eligibility_2'] = eligibility_2
    print("Specimen row is..........")
    print(specimen_row)
    df_specimen.update(specimen_row)

    df_specimen.to_csv('specimen_database3 (4).csv', index=False)


    return 201



In [11]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
import glob

def load_and_preprocess_data(img):
    data = []

    img = keras.preprocessing.image.load_img(img, target_size=(150,150))
    img_array = keras.preprocessing.image.img_to_array(img)


    data = np.array(img_array)

    return data

@app.route('/classifyImage/<int:specimen_ID>', methods=['POST'])
def classifyImage(specimen_ID):
    pattern = f"Specimen_ID_{specimen_ID}.png"  # Define the pattern
    files = glob.glob(f"*{pattern}*")  # Search for files matching the pattern in the current directory

    image = files[0]
    new_image = load_and_preprocess_data(image)
    
    
    model = load_model('breast_cancer_classifier_model.h5')
    
    result = model.predict(np.expand_dims(new_image, axis=0))
    class_names = ["benign", "malignant"]
    prediction = class_names[int(np.round(result[0]))]
    
    df_specimen = pd.read_csv("specimen_database3.csv")


    specimen_row = df_specimen[df_specimen['Specimen_ID'] == specimen_ID]
    specimen_row.loc[specimen_row['Specimen_ID'] == specimen_ID, 'ImageResult'] = prediction 

    df_specimen.update(specimen_row)

    df_specimen.to_csv('specimen_database3.csv', index=False)

    
    specimen_record = df_specimen.loc[df_specimen["Specimen_ID"] == specimen_ID]
    result = specimen_record.to_json(orient="records")
    parsed= loads(result)

    obj = {
        "specimen": parsed
    }
    
    return obj, 201

#obj = classifyImage(5)


2023-11-19 21:24:52.602806: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-19 21:24:52.638457: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-19 21:24:52.639118: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-19 21:24:53.471650: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


<img src="image-20230920-125016.png" width="" align="" />

<img src="image-20230920-125214.png" width="" align="" />

In [12]:
# the following line of code will make this notebook act like a server
app.run(host='0.0.0.0', port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://172.3.10.60:8080
Press CTRL+C to quit
172.3.191.165 - - [19/Nov/2023 21:28:37] "POST /addspecimen HTTP/1.1" 201 -
/shared-libs/python3.11/py/lib/python3.11/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
172.3.40.224 - - [19/Nov/2023 21:28:46] "POST /ArtidisMeasurement/5 HTTP/1.1" 201 -
172.3.40.224 - - [19/Nov/2023 21:28:51] "POST /addeligibility2 HTTP/1.1" 201 -
1/1 [==============================] - 1s 843ms/step
/shared-libs/python3.11/py/lib/python3.11/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWar

### 7. Use Postman to test your endpoints
... and have fun! 😃 💃

<img src="image-20230921-121215.png" width="" align="" />

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=69eb62be-3c1b-49e2-ac1f-d13d4e0bc6d3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>